In [2]:
import csv
import codecs
import fuzzywuzzy
import re
from itertools import groupby
from fuzzywuzzy import process, fuzz
from collections import namedtuple
import requests
import pandas as pd
import numpy as np
import os

In [3]:
def read_csv(file):
    with codecs.open(file, encoding='utf-8') as f:
        data = list(csv.reader(f, delimiter=';', quotechar='"'))[1:]
        return data

In [4]:
catalog = [c for c in read_csv('catalog.txt') if len(c) >= 6 and c[5] == "ru"]
len(catalog)

366289

In [33]:
liter = read_csv('liter.csv')
len(liter)

543

In [34]:
curr_author = None
for l in liter:
    if l[0]:
        curr_author = l[0]
    else:
        l[0] = curr_author

In [7]:
def get_surname(s):
    return re.split(',? ', s)[0]

re.sub(r"(\s?[А-ЯЁ]\.\s?){,2}" + "Чехов" + r"(\s?[А-ЯЁ]\.\s?){,2}", " ", "Ф. М.")

'Ф. М.'

In [8]:
catalog_by_author_surname = dict((surname, list(entries)) for surname, entries in groupby(catalog, lambda x: x[0]))


In [9]:
for surname, books in catalog_by_author_surname.items():
    for b in books:
        b[3] = re.sub(r"(\s?[А-ЯЁ]\.\s?){,2}" + re.escape(surname) + r"(\s?[А-ЯЁ]\.\s?){,2}", " ", b[3])

In [10]:
polka_pairs = [(t, a) for t,a in [["Герой нашего времени","Михаил Лермонтов"],["Анна Каренина","Лев Толстой"],["Мёртвые души","Николай Гоголь"],["Капитанская дочка","Александр Пушкин"],["Война и мир","Лев Толстой"],["Котлован","Андрей Платонов"],["Москва — Петушки","Венедикт Ерофеев"],["Братья Карамазовы","Фёдор Достоевский"],["Идиот","Фёдор Достоевский"],["Петербург","Андрей Белый"],["Преступление и наказание","Фёдор Достоевский"],["Вишнёвый сад","Антон Чехов"],["Конармия","Исаак Бабель"],["Колымские рассказы","Варлам Шаламов"],["Мелкий бес","Фёдор Сологуб"],["Зависть","Юрий Олеша"],["Бесы","Фёдор Достоевский"],["Евгений Онегин","Александр Пушкин"],["Шинель","Николай Гоголь"],["Козлиная песнь","Константин Вагинов"],["Дар","Владимир Набоков"],["Чевенгур","Андрей Платонов"],["Школа для дураков","Саша Соколов"],["Повести Белкина","Александр Пушкин"],["Обломов","Иван Гончаров"],["Мы","Евгений Замятин"],["Ревизор","Николай Гоголь"],["Мастер и Маргарита","Михаил Булгаков"],["Один день Ивана Денисовича","Александр Солженицын"],["Отцы и дети","Иван Тургенев"],["Горе от ума","Александр Грибоедов"],["Нос","Николай Гоголь"],["Двенадцать","Александр Блок"],["Борис Годунов","Александр Пушкин"],["История одного города","Михаил Салтыков-Щедрин"],["Двенадцать стульев","Илья Ильф"],["Приглашение на казнь","Владимир Набоков"],["Доктор Живаго","Борис Пастернак"],["Белая гвардия","Михаил Булгаков"],["Хаджи-Мурат","Лев Толстой"],["Былое и думы","Александр Герцен"],["Чайка","Антон Чехов"],["Смерть Ивана Ильича","Лев Толстой"],["Медный всадник","Александр Пушкин"],["Записки сумасшедшего","Николай Гоголь"],["Маленькие трагедии","Александр Пушкин"],["Соборяне","Николай Лесков"],["Четвёртая проза","Осип Мандельштам"],["Бедная Лиза","Николай Карамзин"],["Записки охотника","Иван Тургенев"],["Старуха","Даниил Хармс"],["Пушкинский дом","Андрей Битов"],["Защита Лужина","Владимир Набоков"],["Жизнь и судьба","Василий Гроссман"],["Господа Головлёвы","Михаил Салтыков-Щедрин"],["Тёмные аллеи","Иван Бунин"],["Житие протопопа Аввакума","Аввакум Петров"],["Невский проспект","Николай Гоголь"],["Левша","Николай Лесков"],["Жизнь Арсеньева","Иван Бунин"],["На дне","Максим Горький"],["Поэма без героя","Анна Ахматова"],["Василий Тёркин","Александр Твардовский"],["Сандро из Чегема","Фазиль Искандер"],["Дом на набережной","Юрий Трифонов"],["Очарованный странник","Николай Лесков"],["Степь","Антон Чехов"],["Золотой телёнок","Евгений Петров"],["Архипелаг ГУЛАГ","Александр Солженицын"],["Факультет ненужных вещей","Юрий Домбровский"],["Пиковая дама","Александр Пушкин"],["Портрет","Николай Гоголь"],["Демон","Михаил Лермонтов"],["Дама с собачкой","Антон Чехов"],["Три сестры","Антон Чехов"],["Леди Макбет Мценского уезда","Николай Лесков"],["Одесские рассказы","Исаак Бабель"],["Лолита","Владимир Набоков"],["Записки из подполья","Фёдор Достоевский"],["Призрак Александра Вольфа","Гайто Газданов"],["Слово о полку Игореве","в спискеДревняя Русь"],["Норма","Владимир Сорокин"],["Картины прошедшего","Александр Сухово-Кобылин"],["Дворянское гнездо","Иван Тургенев"],["Тихий Дон","Михаил Шолохов"],["Город Эн","Леонид Добычин"],["Старосветские помещики","Николай Гоголь"],["Облако в штанах","Владимир Маяковский"],["Случаи","Даниил Хармс"],["Прощание с Матёрой","Валентин Распутин"],["Заповедник","Сергей Довлатов"],["Детство. Отрочество. Юность","Лев Толстой"],["Севастопольские рассказы","Лев Толстой"],["Бедные люди","Фёдор Достоевский"],["Кому на Руси жить хорошо","Николай Некрасов"],["Зангези","Велимир Хлебников"],["Шум времени","Осип Мандельштам"],["Голубая книга","Михаил Зощенко"],["Чапаев и Пустота","Виктор Пелевин"],["Вечера на хуторе близ Диканьки","Николай Гоголь"],["Путешествие из Петербурга в Москву","Александр Радищев"],["Цыганы","Александр Пушкин"],["В овраге","Антон Чехов"],["Опавшие листья","Василий Розанов"],["Ёлка у Ивановых","Александр Введенский"],["Во сне ты горько плакал","Юрий Казаков"],["Поэма Горы","Марина Цветаева"],["Время ночь","Людмила Петрушевская"]]]


In [35]:
def match_catalog_entry(surname, book):
    cat_by_author = catalog_by_author_surname.get(surname)
    if not cat_by_author:
        print("strange surname " + surname)
        return []
        
    cat_book_names = [x[3] for x in cat_by_author]
    best_book, score = process.extractOne(book, cat_book_names, scorer=fuzz.token_sort_ratio)
    if score <= 90:
        return []
    
    best_cat_entry = next(x for x in cat_by_author if x[3] == best_book)
    return best_cat_entry

cat_columns = ["cat_f", "cat_i", "cat_o", "cat_title", "cat_comment", "cat_lang", "cat_issue_year", "cat_collection", "cat_id"]
matched_with_mukhin = pd.DataFrame([match_catalog_entry(get_surname(fio), book) for fio, wiki1, wiki2, book, *rest in liter], columns=cat_columns)
matched_with_polka = pd.DataFrame([match_catalog_entry(fi.split(" ")[1], book) for book, fi in polka_pairs], columns=cat_columns)

strange surname спискеДревняя


In [36]:
mukhin_liter = pd.DataFrame(liter, columns=["mukhin_fio", "mukhin_wiki1", "mukhin_libru", "mukhin_title", "mukhin_year", "mukhin_wiki2"])
with_mukhin = pd.concat([matched_with_mukhin, mukhin_liter], axis=1)
#with_mukhin.cat_id = with_mukhin.cat_id.astype("str")
unmatched_mukhin = with_mukhin[with_mukhin.cat_id.isnull()]
with_mukhin = with_mukhin[with_mukhin.cat_id.notnull()]

with_polka = pd.concat([matched_with_polka, pd.DataFrame(polka_pairs, columns=["polka_title", "polka_fi"])], axis=1)
#with_polka.cat_id = with_polka.cat_id.astype("str")
unmatched_polka = with_polka[with_polka.cat_id.isnull()]
with_polka = with_polka[with_polka.cat_id.notnull()]


In [37]:
manually_selected_cat_ids = set(map(str, [47023, 186062, 53076, 336019, 143082, 402759, 146717, 102649, 18565, 168592, 93198]))
manually_selected_cat = pd.DataFrame([c for c in catalog if c[-1] in manually_selected_cat_ids], columns=cat_columns)
manually_selected_cat

,cat_f,cat_i,cat_o,cat_title,cat_comment,cat_lang,cat_issue_year,cat_collection,cat_id
0,Белый,Андрей,,Серебряный голубь,,ru,1909,,93198
1,Вагинов,Константин,Константинович,Бамбочада,,ru,2008,,168592
2,Газданов,Гайто,Иванович,Пробуждение,,ru,,,18565
3,Герцен,Александр,Иванович,Кто виноват?,,ru,1948,,102649
4,Казаков,Юрий,Павлович,Голубое и зеленое,,ru,1963,,146717
5,Карамзин,Николай,Михайлович,Рыцарь нашего времени,,ru,1803,,402759
6,Лермонтов,Михаил,Юрьевич,Княгиня Лиговская,,ru,,Проза Михаила Лермонтова 0,143082
7,Радищев,Александр,Николаевич,Дневник одной недели,,ru,,,336019
8,Розанов,Василий,Васильевич,Черный огонь,,ru,,,47023
9,Соколов,Саша,,Палисандрия,,ru,,,53076


In [38]:
def get_best_polka_book(author_fio, title):
    polka_of_author = [(name, fi)
                              for name, fi in polka_pairs 
                              if author_fio.find(fi.split(" ")[1]) >= 0]
    polka_titles = [name for name, fi in polka_of_author]
    #title = manual_mukhin_to_polka_correspondences.get(title) or title
    extracted = process.extractOne(title, polka_titles, scorer=fuzz.token_sort_ratio) or (None, 0)
    best_book, score = extracted
    if score > 90:
        return list(next(filter(lambda p: p[0] == best_book, polka_of_author)))
    return [None, None]

unmatched_mukhin_with_polkas = pd.DataFrame(
    [[fio, title] + get_best_polka_book(fio, title) for fio, title in unmatched_mukhin[["mukhin_fio", "mukhin_title"]].values], 
    columns=["mukhin_fio", "mukhin_title", "polka_title", "polka_fi"])
unmatched_polka_not_mentioned_in_mukhin = pd.DataFrame([[title, fi] for title, fi 
                                           in unmatched_polka[["polka_title", "polka_fi"]].values 
                                           if len(unmatched_mukhin_with_polkas[
                                               (unmatched_mukhin_with_polkas.polka_fi == fi) 
                                               & (unmatched_mukhin_with_polkas.polka_title == title)
                                           ]) == 0
                                          ], columns=["polka_title", "polka_fi"])

joined = with_mukhin.set_index(cat_columns).join(with_polka.set_index(cat_columns), how="outer").reset_index()
joined.loc[joined.cat_title == "Преступление и наказание", "cat_id"] = '261797'
joined.loc[joined.cat_title == "Анна Каренина", "cat_id"] = '416697'
joined.loc[joined.polka_title == "Петербург"]
joined.loc[joined.polka_title == "Горе от ума", "cat_id"] = '397134'
joined.loc[joined.polka_title == "Детство. Отрочество. Юность", "cat_id"] = '535985'
joined.loc[joined.polka_title == "Капитанская дочка", "cat_id"] = '276903'
joined.loc[joined.polka_title == "Обломов", "cat_id"] = '485060'
joined.loc[joined.polka_title == "Отцы и дети", "cat_id"] = '499439'
joined.loc[joined.polka_title == "Чапаев и пустота", "cat_id"] = '42053'
joined.loc[joined.polka_title == "Бедные люди", "cat_id"] = '492140'
joined.loc[joined.polka_title == "Картины прошедшего", "cat_id"] = '147229'


all_lists = pd.concat([joined, unmatched_mukhin_with_polkas, unmatched_polka_not_mentioned_in_mukhin, manually_selected_cat]).reset_index()
all_lists.loc[all_lists.polka_title == "Житие протопопа Аввакума", "cat_id"] = '472658'
all_lists.loc[all_lists.polka_title == "Слово о полку Игореве", "cat_id"] = '150566'
all_lists.loc[all_lists.polka_title == "Опавшие листья", "cat_id"] = '175663'
all_lists.loc[all_lists.polka_title == "Ёлка у Ивановых", "cat_id"] = 'elka'

In [40]:
for c in ["mukhin_fio", "cat_f", "cat_i", "cat_o", "polka_fi"]:
    all_lists[c] = all_lists[c].str.replace(",", "").str.replace("ё", "е")
all_lists["some_f_polka"] = np.where(
    all_lists.cat_f.isnull() & all_lists.polka_fi.notnull(), 
    all_lists.polka_fi.str.split(" ").str.get(-1) + " " + all_lists.polka_fi.str.split(" ").str.get(0), 
    all_lists.cat_f + " " + all_lists.cat_i)
all_lists["fi"] = np.where(
    all_lists.some_f_polka.isnull() & all_lists.mukhin_fio.notnull(), 
    all_lists.mukhin_fio.str.split(" ").str.get(0) + " " + all_lists.mukhin_fio.str.split(" ").str.get(1),
    all_lists.some_f_polka)
all_lists = all_lists.drop("some_f_polka", axis=1)
for_grouping = all_lists
set(all_lists.fi)

{'Абрамов Федор',
 'Аверченко Аркадий',
 'Айтматов Чингиз',
 'Аксенов Василий',
 'Акунин Борис',
 'Алексиевич Светлана',
 'Алексин Анатолий',
 'Алешковский Петр',
 'Алешковский Юз',
 'Андреев Даниил',
 'Андреев Леонид',
 'Арсеньев Владимир',
 'Астафьев Виктор',
 'Ахматова Анна',
 'Бабель Исаак',
 'Бажов Павел',
 'Белов Василий',
 'Белый Андрей',
 'Беляев Александр',
 'Бестужев-Марлинский Александр',
 'Битов Андрей',
 'Блок Александр',
 'Бондарев Юрий',
 'Булгаков Михаил',
 'Булычев Кир',
 'Бунин Иван',
 'Быков Василь',
 'Быков Дмитрий',
 'Вагинов Константин',
 'Вайнер Аркадий',
 'Васильев Борис',
 'Введенский Александр',
 'Веллер Михаил',
 'Водолазкин Евгений',
 'Войнович Владимир',
 'Воробьев Константин',
 'Газданов Гайто',
 'Гайдар Аркадий',
 'Гарин-Михайловский Николай',
 'Гаршин Всеволод',
 'Геласимов Андрей',
 'Герцен Александр',
 'Гладков Федор',
 'Гоголь Николай',
 'Гончаров Иван',
 'Горький Максим',
 'Гранин Даниил',
 'Грибоедов Александр',
 'Грин Александр',
 'Гришковец Евгени

In [41]:
all_lists.loc[(all_lists.cat_id == '271485') | (all_lists.fi == "Ильф Илья") | (all_lists.fi == "Петров Евгений"), "fi"] = "Ильф и Петров"
all_lists[all_lists.cat_id == '271485']

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,mukhin_fio,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi
308,308,Остап Бендер 2,полная версия,Петров,Евгений,271485,1994,ru,Петрович,Золотой Теленок,Петров Евгений Ильф Илья,,Золотой теленок,,https://ru.wikipedia.org/wiki/%D0%97%D0%BE%D0%...,1931,Евгений Петров,Золотой телёнок,Ильф и Петров


In [22]:
import traceback

def download_file(url):
    r = requests.get(url, stream=True, proxies = {
        "http": "http://177.72.113.245:54937",
        "https": "http://177.72.113.245:54937",
    }, headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    })
    local_filename = "asd" # re.match('.*filename="(.+)".*', r.headers['Content-Disposition'])[1]
    print(local_filename)
    with open('fb2/' + local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush()
    return local_filename

for url in joined.cat_id:
    try:
        download_file('http://flibusta.is/b/%s/fb2' % id)
    except Exception as e:
        traceback.print_exc()

asd
asd
asd


KeyboardInterrupt: 

In [16]:
joined

,cat_f,cat_i,cat_o,cat_title,cat_comment,cat_lang,cat_issue_year,cat_collection,cat_id,mukhin_fio,mukhin_wiki1,mukhin_libru,mukhin_title,mukhin_year,mukhin_wiki2,polka_title,polka_fi
0,Абрамов,Фёдор,Александрович,Братья и сестры,,ru,1982,Пряслины 1,158030,"Абрамов, Федор Александрович",https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...,http://lib.ru/PROZA/ABRAMOW/,Братья и сестры,1958,https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%...,NaN,NaN
1,Абрамов,Фёдор,Александрович,Две зимы и три лета,,ru,1969,Пряслины 2,158033,"Абрамов, Федор Александрович",,,Две зимы и три лета,1968,,NaN,NaN
2,Абрамов,Фёдор,Александрович,Дом,,ru,1984,Пряслины 4,158032,"Абрамов, Федор Александрович",,,Дом,1978,,NaN,NaN
3,Абрамов,Фёдор,Александрович,Пути-перепутья,,ru,1985,Пряслины 3,158036,"Абрамов, Федор Александрович",,,Пути-перепутья,1973,,NaN,NaN
4,Аверченко,Аркадий,Тимофеевич,Подходцев и двое других,,ru,1917,,159231,"Аверченко, Аркадий Тимофеевич",https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...,http://lib.ru/RUSSLIT/AWERCHENKO/,Подходцев и двое других,1917,,NaN,NaN
5,Аверченко,Аркадий,Тимофеевич,Разумная экономия,,ru,,,417321,"Аверченко, Аркадий Тимофеевич",,,Разумная экономия,ок.1912,,NaN,NaN
6,Аверченко,Аркадий,Тимофеевич,Шутка мецената,,ru,2001,,159241,"Аверченко, Аркадий Тимофеевич",,,Шутка мецената,1925,https://ru.wikipedia.org/wiki/%D0%A8%D1%83%D1%...,NaN,NaN
7,Айтматов,Чингиз,,Белый пароход,,ru,1983,,150859,"Айтматов, Чингиз Торекулович",https://ru.wikipedia.org/wiki/%D0%90%D0%B9%D1%...,http://lib.ru/PROZA/AJTMATOW/,Белый пароход,1970,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%...,NaN,NaN
8,Айтматов,Чингиз,,"Пегий пес, бегущий краем моря",,ru,,,65797,"Айтматов, Чингиз Торекулович",,,"Пегий пес, бегущий краем моря",1977,https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D0%...,NaN,NaN
9,Айтматов,Чингиз,,Плаха,,ru,,,65796,"Айтматов, Чингиз Торекулович",,,Плаха,1986,https://ru.wikipedia.org/wiki/%D0%9F%D0%BB%D0%...,NaN,NaN


In [15]:
import os

books = ['fb2/' + f for f in os.listdir('fb2') if f[-7:] == "fb2.zip"]
len(books)


451

In [200]:
import zipfile
for b in books:
    zip_ref = zipfile.ZipFile(b, 'r')
    zip_ref.extractall('fb2.unzipped')
    zip_ref.close()

In [201]:
import subprocess
subprocess.call(['powershell.exe', '-File', 'fb2_2_txt.ps1'])

0

In [202]:
import subprocess
import os
import time

processes = set()
max_processes = 5

for f in os.listdir('txt'):
    if (f[-3:] == "txt"):
        json_file_name = "json/" + f + ".json"
        if not os.path.isfile(json_file_name):
            processes.add(subprocess.Popen(["mystem", "--format=json", "-nigfcsd", "txt/" + f, json_file_name]))
            while len(processes) >= max_processes:
                time.sleep(0.2)
                processes.difference_update(
                    [p for p in processes if p.poll() is not None])
            print("creating", json_file_name)
        else:
            print("exists", json_file_name)
    
#Check if all the child processes were closed
for p in processes:
    if p.poll() is None:
        p.wait()


exists json/100333.fb2.txt.json
exists json/101185.fb2.txt.json
exists json/102112.fb2.txt.json
exists json/102645.fb2.txt.json
creating json/102649.fb2.txt.json
exists json/103387.fb2.txt.json
exists json/103588.fb2.txt.json
exists json/10547.fb2.txt.json
exists json/105610.fb2.txt.json
exists json/105764.fb2.txt.json
exists json/1061.fb2.txt.json
exists json/1068.fb2.txt.json
exists json/1070.fb2.txt.json
exists json/107678.fb2.txt.json
exists json/107708.fb2.txt.json
exists json/109209.fb2.txt.json
exists json/110929.fb2.txt.json
exists json/111375.fb2.txt.json
exists json/1122.fb2.txt.json
exists json/113370.fb2.txt.json
exists json/113372.fb2.txt.json
exists json/114299.fb2.txt.json
exists json/114591.fb2.txt.json
exists json/115721.fb2.txt.json
exists json/115826.fb2.txt.json
exists json/119564.fb2.txt.json
exists json/122412.fb2.txt.json
exists json/123391.fb2.txt.json
exists json/124468.fb2.txt.json
exists json/124470.fb2.txt.json
exists json/124471.fb2.txt.json
exists json/125

exists json/312454.fb2.txt.json
exists json/312635.fb2.txt.json
exists json/313626.fb2.txt.json
exists json/315588.fb2.txt.json
exists json/315732.fb2.txt.json
exists json/320065.fb2.txt.json
exists json/320174.fb2.txt.json
exists json/322391.fb2.txt.json
exists json/323998.fb2.txt.json
exists json/324990.fb2.txt.json
exists json/329591.fb2.txt.json
exists json/330813.fb2.txt.json
exists json/331966.fb2.txt.json
exists json/334920.fb2.txt.json
creating json/336019.fb2.txt.json
exists json/340699.fb2.txt.json
exists json/343914.fb2.txt.json
exists json/347944.fb2.txt.json
exists json/348600.fb2.txt.json
exists json/348601.fb2.txt.json
exists json/348602.fb2.txt.json
exists json/348800.fb2.txt.json
exists json/348954.fb2.txt.json
exists json/350166.fb2.txt.json
exists json/351632.fb2.txt.json
exists json/35517.fb2.txt.json
exists json/358535.fb2.txt.json
exists json/359870.fb2.txt.json
exists json/36282.fb2.txt.json
exists json/365436.fb2.txt.json
exists json/365649.fb2.txt.json
exists j

In [42]:
def get_id_by_file(f):
    return f.split("/")[-1].split(".")[0]
downloaded_ids = { get_id_by_file(f) for f in os.listdir("txt") }
all_lists[(all_lists.cat_id.isnull() | ~all_lists.cat_id.isin(downloaded_ids)) & all_lists.polka_title.notnull()]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,mukhin_fio,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi
599,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Велимир Хлебников,Зангези,Хлебников Велимир


In [43]:
author_downloaded_counts = for_grouping[for_grouping.cat_id.notnull() & for_grouping.cat_id.isin(downloaded_ids)].fi.value_counts()
author_downloaded_counts = pd.DataFrame(list(author_downloaded_counts.items()), columns=["f", "book_ct"])
single_authors = list(author_downloaded_counts[author_downloaded_counts.book_ct == 1].f)
for_grouping[for_grouping.fi.isin(single_authors)]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,mukhin_fio,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi
37,37,,,Ахматова,Анна,134319,2007,ru,Андреевна,Поэма без героя,NaN,NaN,NaN,NaN,NaN,NaN,Анна Ахматова,Поэма без героя,Ахматова Анна
46,46,Отцы основатели - русское пространство. Алекса...,,Беляев,Александр,245747,2010,ru,Романович,Голова профессора Доуэля,Беляев Александр Романович,http://lib.ru/RUFANT/BELAEW/,Голова профессора Доуэля,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%...,https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D0%...,1923 - 1928,NaN,NaN,Беляев Александр
47,47,,,Беляев,Александр,409106,1994,ru,Романович,Остров погибших кораблей,Беляев Александр Романович,,Остров погибших кораблей,,https://ru.wikipedia.org/wiki/%D0%9E%D1%81%D1%...,1923 - 1928,NaN,NaN,Беляев Александр
48,48,Отцы основатели - русское пространство. Алекса...,,Беляев,Александр,245745,2009,ru,Романович,Продавец воздуха,Беляев Александр Романович,,Продавец воздуха,,https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%...,ок.1929,NaN,NaN,Беляев Александр
49,49,,,Беляев,Александр,145500,2001,ru,Романович,"Человек, потерявший лицо",Беляев Александр Романович,,"Человек, потерявший лицо",,https://ru.wikipedia.org/wiki/%D0%A7%D0%B5%D0%...,1923 - 1928,NaN,NaN,Беляев Александр
50,50,Отцы основатели - русское пространство. Алекса...,,Беляев,Александр,245744,2009,ru,Романович,Человек-амфибия,Беляев Александр Романович,,Человек-амфибия,,https://ru.wikipedia.org/wiki/%D0%A7%D0%B5%D0%...,1927,NaN,NaN,Беляев Александр
56,56,Мировая классика 0,,Блок,Александр,417985,2008,ru,Александрович,Двенадцать,NaN,NaN,NaN,NaN,NaN,NaN,Александр Блок,Двенадцать,Блок Александр
65,65,Театр теней 1,,Булычев,Кир,108330,2005,ru,,Вид на битву с высоты,Булычев Кир,,Вид на битву с высоты,,,1998,NaN,NaN,Булычев Кир
66,66,Великий Гусляр 0|Классическая библиотека прикл...,сборник,Булычев,Кир,408502,2015,ru,,Перпендикулярный мир,Булычев Кир,,Перпендикулярный мир,,,1989,NaN,NaN,Булычев Кир
67,67,Отцы-основатели: русское пространство. Кир Бул...,сборник,Булычев,Кир,245767,2005,ru,,Похищение чародея,Булычев Кир,,Похищение чародея,,,1979,NaN,NaN,Булычев Кир


In [44]:
from pathlib import Path


def get_non_prose_metrics(file_path):
    lines = [l for l in Path(file_path).read_text(encoding="utf8").split("\n") if l]
    descr = all_lists[all_lists.cat_id == get_id_by_file(file_path)].iloc[0]
    return (
        descr["fi"],
        np.nan_to_num(descr["cat_title"]) or np.nan_to_num(descr["mukhin_title"]) or np.nan_to_num(descr["polka_title"]),
        lines[len(lines) // 2 : len(lines) // 2 + 10],
        np.average([len(l) for l in lines]),
        np.average([len(l.split(".")[0]) for l in lines])
    )

In [21]:
poem_ids = []
poem_or_play_ids = []
for txt_path in os.listdir("txt"):
    f, title, sample, avg_line_len, avg_first_sent_len = get_non_prose_metrics("txt/" + txt_path)
    if avg_line_len < 50:
        poem_ids.append(get_id_by_file(txt_path))
        #print(str(len(poem_ids)) + ".", f, title)
        #print("\n".join(sample), "\n________\n")
    if avg_first_sent_len < 30:
        poem_or_play_ids.append(get_id_by_file(txt_path))

In [22]:
for_grouping[
    for_grouping.fi.isin(single_authors) & 
    for_grouping.cat_id.isin(downloaded_ids) & 
    (for_grouping.cat_id.isin(poem_ids) |
    for_grouping.cat_id.isin(poem_or_play_ids))
][["polka_fi", "polka_title"]]

,polka_fi,polka_title
37,Анна Ахматова,Поэма без героя
56,Александр Блок,Двенадцать
146,Александр Грибоедов,Горе от ума
267,Владимир Маяковский,Облако в штанах
286,Николай Некрасов,Кому на Руси жить хорошо
404,Александр Сухово-Кобылин,Картины прошедшего
405,Александр Твардовский,Василий Тёркин
460,Марина Цветаева,Поэма Горы
598,в спискеДревняя Русь,Слово о полку Игореве
601,Александр Введенский,Ёлка у Ивановых


In [ ]:
for poem_id in set(poem_ids) - set(poem_or_play_ids):
    print(poem_id, get_non_prose_metrics("txt/" + poem_id + ".fb2.txt"))

In [45]:
for_grouping["poem_or_play"] = pd.to_numeric(for_grouping.cat_id.isin(poem_ids + poem_or_play_ids) | (for_grouping.polka_title == "Зангези"))
for_grouping["not_poem_or_play"] = ~for_grouping.poem_or_play
with_poem_count = for_grouping.groupby("fi").agg({ "poem_or_play": sum, "not_poem_or_play": sum }).reset_index()

for_grouping[for_grouping.polka_title == "Зангези"]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,...,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi,poem_or_play,not_poem_or_play
599,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Велимир Хлебников,Зангези,Хлебников Велимир,True,False


In [46]:
polka_authors = set(for_grouping[for_grouping.polka_title.notnull()].fi)
print(len(polka_authors))
polka_authors
with_not_enough_prose_by_polka_authors = with_poem_count[with_poem_count.fi.isin(polka_authors) & (with_poem_count.not_poem_or_play < 2) & (with_poem_count.not_poem_or_play > 0)]
with_not_enough_prose_by_polka_authors

60


,fi,poem_or_play,not_poem_or_play
102,Петров Аввакум,0.0,1.0


In [47]:
for_grouping[for_grouping.fi.isin(with_not_enough_prose_by_polka_authors.fi) & for_grouping.not_poem_or_play]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,...,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi,poem_or_play,not_poem_or_play
597,0,NaN,NaN,NaN,NaN,472658,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Аввакум Петров,Житие протопопа Аввакума,Петров Аввакум,False,True


In [48]:
all_lists["poem_or_play"] = all_lists.cat_id.isin(poem_ids + poem_or_play_ids) | (for_grouping.polka_title == "Зангези")
all_lists["downloaded"] = all_lists.cat_id.isin(downloaded_ids)
#all_lists[all_lists.cat_id]
all_lists.to_csv("selected_books.csv", encoding="utf-8-sig")

In [29]:
import collections



,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,...,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi,poem_or_play,not_poem_or_play,downloaded
200,200,Остап Бендер 1,«Каноническое» издание,Ильф,Илья,105610,1987,ru,Арнольдович,Двенадцать стульев,...,Двенадцать стульев,https://ru.wikipedia.org/wiki/%D0%98%D0%BB%D1%...,https://ru.wikipedia.org/wiki/%D0%94%D0%B2%D0%...,1927,Илья Ильф,Двенадцать стульев,Ильф Илья,False,True,True
201,201,Остап Бендер 2,полная версия,Ильф,Илья,271485,1994,ru,Арнольдович,Золотой Теленок,...,Золотой теленок,,https://ru.wikipedia.org/wiki/%D0%97%D0%BE%D0%...,1931,NaN,NaN,Ильф Илья,False,True,True
202,202,,,Ильф,Илья,168602,2008,ru,Арнольдович,Одноэтажная Америка,...,Одноэтажная Америка,,https://ru.wikipedia.org/wiki/%D0%9E%D0%B4%D0%...,1936,NaN,NaN,Ильф Илья,False,True,True
309,309,Остап Бендер 2,полная версия,Петров,Евгений,271485,1994,ru,Петрович,Золотой Теленок,...,NaN,NaN,NaN,NaN,Евгений Петров,Золотой телёнок,Петров Евгений,False,True,True
